# Part 2: Perform the MCMC Density Fits

This notebook is Part 2 of the code for reproducing [Imig et al 2025](https://astrojimig.github.io/pdfs/Imig_MW_density.pdf). In this notebook, the effective selection function calcualted in Part 1 is applied to fit the density profile of each stellar population with an MCMC likelihood optimization.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as fits
import os

from mw_density import sample_selection, plotting_helpers, mcmc_functions
from mw_density.selection_function import SelectionFunction

sample_selection.set_env_variables()

In [ ]:
# Define Plotting Parameters
params = {
    "axes.labelsize": 36,
    "xtick.labelsize": 36,
    "ytick.labelsize": 36,
    "text.usetex": False,
    "lines.linewidth": 1,
    "axes.titlesize": 22,
    "font.family": "serif",
    "font.size": 36,
}
plt.rcParams.update(params)

In [ ]:
# Unique ID for saving files
unique_id = "4-22-25"

## 2A: Data Set Up

### Load in Files

In [ ]:
# ===========================
# Load in Selection Function
# ===========================
selfunc = SelectionFunction()

### Sample Definition

In [ ]:
age_bins, mh_bins = sample_selection.setup_maap_bins()
apogee_sample = fits.open("data/apogee_sample.fits")[1].data

logg_min_lim = np.min(apogee_sample["LOGG"])
logg_max_lim = np.max(apogee_sample["LOGG"])

metals = apogee_sample["M_H"]
alphas = apogee_sample["ALPHA_M"]
ages = apogee_sample["AGE"]
data_rs = apogee_sample["GALACTIC_R"]
data_zs = apogee_sample["GALACTIC_Z"]

In [ ]:
print(f"Total Number of stars in Sample: {len(metals)}")

### Count how many stars are in each stellar population bin

In [ ]:
ncount_distmass_high = np.zeros(
    (len(age_bins["center"]), len(mh_bins["center"]))
)
ncount_distmass_low = np.zeros(
    (len(age_bins["center"]), len(mh_bins["center"]))
)

tot = len(age_bins["center"]) * len(mh_bins["center"])

low_alph_mask, high_alph_mask = sample_selection.get_alpha_masks(apogee_sample)

cnter = 0
for i_a, age in enumerate(age_bins["center"]):
    for i_m, mh in enumerate(mh_bins["center"]):
        m = (apogee_sample["AGE_BIN_I"] == i_a) & (
            apogee_sample["METAL_BIN_I"] == i_m
        )
        hm = m & high_alph_mask
        lm = m & low_alph_mask
        ncount_distmass_high[i_a, i_m] = len(hm[hm])
        ncount_distmass_low[i_a, i_m] = len(lm[lm])

        cnter += 1
        # print('{}/{}'.format(cnter,tot))

In [ ]:
plotting_helpers.bin_count_plot_histo(
    ncount_distmass_low, ncount_distmass_high
)

In [ ]:
print(len(ncount_distmass_low.flatten()[ncount_distmass_low.flatten() > 100]))

print(
    len(ncount_distmass_high.flatten()[ncount_distmass_high.flatten() > 100])
)

print(
    len(ncount_distmass_low.flatten()[ncount_distmass_low.flatten() > 100])
    + len(ncount_distmass_high.flatten()[ncount_distmass_high.flatten() > 100])
)


## 2C: Perform the MCMC fits

This generally takes a few minutes for each bin.

In [ ]:
# Set to FALSE if you want to preserve progress
overwrite_files = False

In [ ]:
# High alpha bins
bin_counter = 0
N_bins = len(mh_bins["center"]) * len(age_bins["center"])

# TESTBIN = 39
# repeat_bins = [5,25,27,50,74] # select specific bins to re-run
repeat_bins = []

for i_m, mh in enumerate(mh_bins["center"]):
    for i_a, age in enumerate(age_bins["center"]):
        # Log basic data
        print("=" * 50)
        print(f"Bin {bin_counter + 1}/{N_bins}")
        fname = f"results/mcmc_chains/bin{bin_counter}_{unique_id}_high.npz"
        # Check if file exists:
        if not os.path.exists(fname) or (overwrite_files is True):
            # Get effsel for this bin
            bin_effsel = selfunc.effsel[bin_counter]
            effsel_mask = (bin_effsel.flatten() != 0) & (
                np.isfinite(bin_effsel.flatten())
            )
            bin_effsel_rs = selfunc.coordinates["r"][effsel_mask]
            bin_effsel_zs = selfunc.coordinates["z"][effsel_mask]
            bin_effsel = bin_effsel.flatten()[effsel_mask]
            eff_volume = selfunc.calc_eff_survey_volume(bin_counter)

            effsel_dict = {
                "bin_effsel": bin_effsel,
                "bin_effsel_rs": bin_effsel_rs,
                "bin_effsel_zs": bin_effsel_zs,
                "eff_volume": np.array(eff_volume[1])[effsel_mask],
            }

            # Run MCMC
            mcmc_functions.perform_maap_density_fit(
                apogee_sample, effsel_dict, i_m, i_a, "HIGH", fname, nthreads=8
            )

        else:  # File aready exists.
            print(
                f"File {fname} already exists. Skipping bin. "
                "Set overwrite_files=True to override)"
            )

        bin_counter += 1


In [ ]:
# Low alpha bins
bin_counter = 0
N_bins = len(mh_bins["center"]) * len(age_bins["center"])

TESTBIN = 39
# repeat_bins = [5,25,27,50,74] #bins with effsel problem
repeat_bins = []

for i_m, mh in enumerate(mh_bins["center"]):
    for i_a, age in enumerate(age_bins["center"]):
        # Log basic data
        print("=" * 50)
        print(f"Bin {bin_counter + 1}/{N_bins}")

        fname = f"results/mcmc_chains/bin{bin_counter}_{unique_id}.npz"
        # Check if file exists:
        if not os.path.exists(fname) or (overwrite_files is True):
            # Get effsel for this bin
            bin_effsel = selfunc.effsel[bin_counter]
            effsel_mask = (bin_effsel.flatten() != 0) & (
                np.isfinite(bin_effsel.flatten())
            )
            bin_effsel_rs = selfunc.coordinates["r"][effsel_mask]
            bin_effsel_zs = selfunc.coordinates["z"][effsel_mask]
            bin_effsel = bin_effsel.flatten()[effsel_mask]
            eff_volume = selfunc.calc_eff_survey_volume(bin_counter)

            effsel_dict = {
                "bin_effsel": bin_effsel,
                "bin_effsel_rs": bin_effsel_rs,
                "bin_effsel_zs": bin_effsel_zs,
                "eff_volume": np.array(eff_volume[1])[effsel_mask],
            }
            # Run MCMC
            mcmc_functions.perform_maap_density_fit(
                apogee_sample, effsel_dict, i_m, i_a, "LOW", fname, nthreads=8
            )
        else:  # File aready exists.
            print(
                f"File {fname} already exists. Skipping bin."
                " (Set overwrite_files=True to override)"
            )

        bin_counter += 1


The best-fit parameters are saved out as a fits file in `make_results_file.ipynb`!